<a href="https://colab.research.google.com/github/Hyun-ho-Lee/Graduate-school/blob/main/deeping_%EC%BD%94%EB%93%9C_%EB%B6%84%EC%84%9D_08_14_ipnynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import datetime
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import statsmodels.api as sm

### Date and Author: 20201223, Kyungwon Kim ###

target_KRX = ['삼성전자', 'SK하이닉스', '셀트리온', '삼성SDI', '현대자동차', '기아자동차', '포스코']
#df = get_data_from_Yahoo(target_KRX)
#def get_data_from_Yahoo(name_list, start='2013-12-01', end='2018-12-31', same_length=False):
 

name_list=target_KRX
start='2015-12-01'
end='2020-12-31'
start = pd.to_datetime(start)
end = pd.to_datetime(end)
same_length=False
#@title
## 야후에서 KOSPI 지수 KODAQ 지수 크롤링해와서 KOSPI 주식 이름 가져오는 과정 
df_code_kospi = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&marketType=stockMkt', header=0)[0][['회사명', '종목코드']]
df_code_kospi.rename(columns={'회사명': 'Name', '종목코드': 'Code'}, inplace=True)
df_code_kospi.Code = df_code_kospi.Code.map('{:06d}.KS'.format)
df_code_kosdaq = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&marketType=kosdaqMkt', header=0)[0][['회사명', '종목코드']]
df_code_kosdaq.rename(columns={'회사명': 'Name', '종목코드': 'Code'}, inplace=True)
df_code_kosdaq.Code = df_code_kosdaq.Code.map('{:06d}.KQ'.format)
df_code = pd.concat([df_code_kospi, df_code_kosdaq], axis=0).reset_index().iloc[:,1:]
code_KRX = list(df_code['Name'].values)

# code extraction name list 에 맞춰 kospi 종목 코드를 code_list에 저장하는 과정
code_list = []
for name in tqdm(name_list):
   if name in code_KRX:
         code = df_code.query("Name=='{}'".format(name))['Code'].to_string(index=False).strip()
         code_list.append(code)   

# data extraction nmae  code_list에 있는 kospi 종목들 종가를 dict_final 이라는 dictionary 에 저장하는 과정 key: value 값 형태 
dict_final = dict()
for i, name in enumerate(tqdm(code_list)):
    df_name = pdr.data.DataReader(name, 'yahoo', start, end)[['Close']]
    df_name = df_name.dropna()
    df_name.columns = [name]
    dict_final[name_list[i]] = df_name

    # data filtering with same length
#인덱스에 date (일별) 그리고 변수 이름이 주식 코드이며 종가로 데이터 프레임화 시키는 과정인거 같지만 same_length가 무엇인지 모르겠음 

#또한 timedelta 라는게 2018-12-31 - 2013-12-01 인데 그 값이 1800이상이며 dict_final[key].shape[0]보다 크므로 if 문 자체가 안돌아감 

#여기서 dict_final[key].shape[0]은 dict row 갯수임 

timedelta = (pd.to_datetime(end)-pd.to_datetime(start)).days + 1
df_samelength = pd.DataFrame()
target_list = []
if same_length:
    for key, val in dict_final.items():
        if dict_final[key].shape[0] >= timedelta:
          target_list.append(key)
          df_samelength = pd.concat([df_samelength, dict_final[key].iloc[-timedelta:].reset_index().iloc[:,1:]], axis=1) #pd.concat의 경우 axis=1 으로 지정이므로 왼쪽+ 오른쪽으로 합치는 과정 
    df_samelength.index = pd.date_range(start, periods=timedelta) #periods time delta 때문에 오류뜸 




100%|██████████| 6/6 [00:03<00:00,  1.89it/s]


In [ ]:
timedelta


1858

In [ ]:
#@title
## 야후에서 KOSPI 지수 KODAQ 지수 크롤링해와서 KOSPI 주식 이름 가져오는 과정 
df_code_kospi = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&marketType=stockMkt', header=0)[0][['회사명', '종목코드']]
df_code_kospi.rename(columns={'회사명': 'Name', '종목코드': 'Code'}, inplace=True)
df_code_kospi.Code = df_code_kospi.Code.map('{:06d}.KS'.format)
df_code_kosdaq = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&marketType=kosdaqMkt', header=0)[0][['회사명', '종목코드']]
df_code_kosdaq.rename(columns={'회사명': 'Name', '종목코드': 'Code'}, inplace=True)
df_code_kosdaq.Code = df_code_kosdaq.Code.map('{:06d}.KQ'.format)
df_code = pd.concat([df_code_kospi, df_code_kosdaq], axis=0).reset_index().iloc[:,1:]
code_KRX = list(df_code['Name'].values)

In [ ]:
df_code

NameError: ignored

In [ ]:
# code extraction name list 에 맞춰 kospi 종목 코드를 code_list에 저장하는 과정
code_list = []
for name in tqdm(name_list):
   if name in code_KRX:
         code = df_code.query("Name=='{}'".format(name))['Code'].to_string(index=False).strip()
         code_list.append(code)   

100%|██████████| 8/8 [00:00<00:00, 214.75it/s]


In [ ]:
code_list

['005930.KS',
 '000660.KS',
 '068270.KS',
 '006400.KS',
 '005380.KS',
 '005490.KS',
 '015760.KS']

In [ ]:
# data extraction nmae  code_list에 있는 kospi 종목들 종가를 dict_final 이라는 dictionary 에 저장하는 과정 key: value 값 형태 
dict_final = dict()
for i, name in enumerate(tqdm(code_list)):
    df_name = pdr.data.DataReader(name, 'yahoo', start, end)[['Close']]
    df_name = df_name.dropna()
    df_name.columns = [name]
    dict_final[name_list[i]] = df_name
    print(dict_final)

In [ ]:
dict_final

In [ ]:
# data filtering with same length
#인덱스에 date (일별) 그리고 변수 이름이 주식 코드이며 종가로 데이터 프레임화 시키는 과정인거 같지만 same_length가 무엇인지 모르겠음 

#또한 timedelta 라는게 2018-12-31 - 2013-12-01 인데 그 값이 1800이상이며 dict_final[key].shape[0]보다 크므로 if 문 자체가 안돌아감 

#여기서 dict_final[key].shape[0]은 dict row 갯수임 

timedelta = (pd.to_datetime(end)-pd.to_datetime(start)).days + 1
df_samelength = pd.DataFrame()
target_list = []
if same_length:
    for key, val in dict_final.items():
        if dict_final[key].shape[0] >= timedelta:
          target_list.append(key)
          df_samelength = pd.concat([df_samelength, dict_final[key].iloc[-timedelta:].reset_index().iloc[:,1:]], axis=1) #pd.concat의 경우 axis=1 으로 지정이므로 왼쪽+ 오른쪽으로 합치는 과정 
    df_samelength.index = pd.date_range(start, periods=timedelta) #periods time delta 때문에 오류뜸 

return dict_final, df_samelength, target_list

SyntaxError: ignored

In [ ]:
df_samelength = pd.concat([df_samelength, dict_final[key].iloc[-timedelta:].reset_index().iloc[:,1:]], axis=1)
df_samelength

,015760.KS,015760.KS
0,31950.0,31950.0
1,31750.0,31750.0
2,31650.0,31650.0
3,30900.0,30900.0
4,30700.0,30700.0
...,...,...
1236,33450.0,33450.0
1237,33100.0,33100.0
1238,32750.0,32750.0
1239,32500.0,32500.0


In [ ]:
target_list.append(list(dict_final.keys()))

In [ ]:
target_list

[['삼성전자', 'SK하이닉스', '셀트리온', '삼성SDI', '현대자동차', '기아자동차', '포스코']]

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import losses
import tensorflow as tf

In [ ]:
#딥러닝 돌리기 
algorithms = ['MLP', 'RNN', 'GRNN', 'LSTM', 'GRU']

In [ ]:
#X_train 을 어떤걸로 지정해야할지 y_train을 어떤걸로 지정해야할지 감이 안잡힘 

def algorithm_reg_deeplearning(algorithms, X_train, Y_train, X_test, Y_test, scaler,optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],batch_size=64, epochs=10, verbose=0,plot_epoch=False):
    RMSE_train, RMSE_test = algorithm_reg_deeplearning(algorithms, X_train, Y_train, X_test, Y_test,scaler,optimizer='adam', loss='mean_squared_error',batch_size=32, epochs=30, verbose=0,plot_epoch=False) 

In [ ]:
RMSE_train, RMSE_test = algorithm_reg_deeplearning(algorithms, X_train, Y_train, X_test, Y_test,scaler,optimizer='adam', loss='mean_squared_error',batch_size=32, epochs=30, verbose=0,plot_epoch=False)

In [ ]:
def algorithm_reg_deeplearning(algorithms, X_train, Y_train, X_test, Y_test, scaler,
                               optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],
                               batch_size=64, epochs=10, verbose=0,
                               plot_epoch=False):

SyntaxError: ignored

In [ ]:
seed_num = 123
np.random.seed(seed_num)
random.seed(seed_num)
tf.random.set_seed(seed_num)

In [ ]:
RMSE_train, RMSE_test = [], []
for alg in tqdm(algorithms):
        if alg == 'MLP':
            model = Sequential()
            model.add(Dense(128, input_shape=(X_train.shape[1]*X_train.shape[2],), activation='relu'))
            model.add(Dense(256, activation='relu'))
            model.add(Dense(128, activation='relu'))
            model.add(Dense(64, activation='relu'))
            model.add(Dense(1))
            model.compile(optimizer=optimizer, loss=loss)
            early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
            model_fit = model.fit(X_train.reshape(X_train.shape[0], -1), Y_train,
                                  batch_size=batch_size, epochs=epochs,
                                  validation_data=(X_test.reshape(X_test.shape[0], -1), Y_test),
                                  callbacks=[early_stop],
                                  verbose=verbose)
        elif alg == 'RNN':
            model = Sequential()
            model.add(SimpleRNN(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='relu'))
            model.add(SimpleRNN(256, return_sequences=True, activation="relu"))
            model.add(SimpleRNN(128, return_sequences=True, activation="relu"))
            model.add(SimpleRNN(64, return_sequences=True, activation="relu"))
            model.add(Flatten())
            model.add(Dense(1))
            model.compile(optimizer=optimizer, loss=loss)
            early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
            model_fit = model.fit(X_train, Y_train,
                                  batch_size=batch_size, epochs=epochs,
                                  validation_data=(X_test, Y_test),
                                  callbacks=[early_stop],
                                  verbose=verbose)
        elif alg == 'GRNN':
            params = {'kernel':["RBF"],
                      'sigma' : list(np.arange(0.01, 3, 0.01)),
                      'calibration' : ['None']}
            model = GridSearchCV(estimator=GRNN(), param_grid=params,
                                 scoring='neg_mean_squared_error',
                                 cv=10, verbose=verbose)  
            model.fit(X_train.reshape(X_train.shape[0], -1), Y_train)
        elif alg == 'LSTM':
            model = Sequential()
            model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='relu'))
            model.add(LSTM(256, return_sequences=True, activation="relu"))
            model.add(LSTM(128, return_sequences=True, activation="relu"))
            model.add(LSTM(64, return_sequences=False, activation="relu"))
            model.add(Dense(1))
            model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
            early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
            model_fit = model.fit(X_train, Y_train,
                                  batch_size=batch_size, epochs=epochs,
                                  validation_data=(X_test, Y_test),
                                  callbacks=[early_stop],
                                  verbose=verbose)
        elif alg == 'GRU':
            model = Sequential()
            model.add(GRU(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='relu'))
            model.add(GRU(256, return_sequences=True, activation="relu"))
            model.add(GRU(128, return_sequences=True, activation="relu"))
            model.add(GRU(64, return_sequences=False, activation="relu"))
            model.add(Dense(1))
            model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
            early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
            model_fit = model.fit(X_train, Y_train,
                                  batch_size=batch_size, epochs=epochs,
                                  validation_data=(X_test, Y_test),
                                  callbacks=[early_stop],
                                  verbose=verbose)

  0%|          | 0/5 [00:00<?, ?it/s]


NameError: ignored

In [ ]:
# plot of model flow
if plot_epoch:
  if alg != 'GRNN':
          model.summary()
        fontsize = 20
        plt.figsize=(16, 8)
        plt.plot(pd.DataFrame(model_fit.history['loss']), label='Loss: Train')
        plt.plot(pd.DataFrame(model_fit.history['val_loss']), label='Loss: Test')
        plt.xticks(fontsize=fontsize, rotation=0)
        plt.yticks(fontsize=fontsize)
        plt.xlabel('Epoch', fontname='serif', fontsize=fontsize+2)
        plt.legend(fontsize=fontsize-4, loc='best')
        plt.grid(axis='y')
        plt.show()

In [ ]:
#predict

if (alg == 'MLP') | (alg == 'GRNN'):
    Y_train_pred = model.predict(X_train.reshape(X_train.shape[0], -1))
    Y_test_pred = model.predict(X_test.reshape(X_test.shape[0], -1))
else:
    Y_train_pred = model.predict(X_train)
    Y_test_pred = model.predict(X_test)

In [ ]:
#rescaling

if scaler != []:
  if alg == 'GRNN':
    Y_train_orig = scaler.inverse_transform(Y_train.reshape(-1,1))
    Y_train_pred = scaler.inverse_transform(Y_train_pred.reshape(-1,1))
    Y_test_orig = scaler.inverse_transform(Y_test.reshape(-1,1))
    Y_test_pred = scaler.inverse_transform(Y_test_pred.reshape(-1,1))
  else:
    Y_train_orig = scaler.inverse_transform(Y_train)
    Y_train_pred = scaler.inverse_transform(Y_train_pred)
    Y_test_orig = scaler.inverse_transform(Y_test)
    Y_test_pred = scaler.inverse_transform(Y_test_pred)
else:
  Y_train_orig = Y_train.copy()
  Y_test_orig = Y_test.copy()

In [ ]:
#evaluation

RMSE_tr = mean_squared_error(Y_train_pred, Y_train_orig)**0.5
RMSE_te = mean_squared_error(Y_test_pred, Y_test_orig)**0.5
RMSE_train.append(RMSE_tr)
RMSE_test.append(RMSE_te)

In [ ]:
RMSE_train = pd.DataFrame(RMSE_train)
RMSE_train.index = algorithms
RMSE_train.columns = ['Train']
RMSE_test = pd.DataFrame(RMSE_test)
RMSE_test.index = algorithms
RMSE_test.columns = ['Test']
   
return RMSE_train, RMSE_test